# Vaccine Development with Monte Carlo Policy Evaluation

As the CEO of a biotech company which is considering the development of a new vaccine. Starting at phase 0 (state 0), the drug develpment can stay in the same state or advance to "phase 1  with promising results" (state 1) or advance to "phase 1 with disappointing results" (state 2), or fail completely (state 4). At phase 1, the drug can stay in the same state, fail or become a success (state 3), in which case you will sell its patent to a big pharma company for \$10 million.
These state transitions happen from month to month, and at each state, you have the option to make an additional investment of \$100,000, which increases the chances of success.

After careful study, your analysts develop the program below to simulate different scenarios using statistical data from similar projects. For this question, you don't need to understand what is inside the program, only how to use it. If you want, you can can skip to the next cell.

Use the Monte Carlo method to find the value functions induced by the following policies:


a) Always invest

b) Never invest

Use a discount factor of 0.9960 to discount future rewards.



In [ ]:
import numpy as np
class MDP():
  def __init__(self):
    self.A = [0, 1]
    self.S = [0, 1, 2, 3, 4]

    P0 = np.array([[0.5, .15, .15, 0, .20],
                   [0, .5, .0, .25, .25],
                   [0, 0, .15, .05, .8],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R0 = np.array([0, 0, 0, 10, 0])

    P1 = np.array([[0.5, .25, .15, 0, .10],
                   [0, .5, .0, .35, .15],
                   [0, 0, .20, .05, .75],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

    self.P = [P0, P1]
    self.R = [R0, R1]

  def step(self, s, a):
    s_prime = np.random.choice(len(self.S), p=self.P[a][s])
    R = self.R[a][s]
    if s_prime == 4:
      done = True
    else:
      done = False
    return s_prime, R, done

  def simulate(self, s, a, π):
    done = False
    t = 0
    history = []
    while not done:
      if t > 0:
        a = π[s]
      s_prime, R, done = self.step(s, a)
      history.append((s, a, R))
      s = s_prime
      t += 1

    return history

In [ ]:
# Here is how you can use the program
# First we create an instance of this MDP
mdp = MDP()

In [ ]:
# Now, starting at phase 0 (state s=0), we will choose to make additional investments (a=1).
s = 0
a = 1

π = [0, 0, 0, 0]

# We feed the state-action pair to the simulator and it tells us what happened in this simulation:
history = mdp.simulate(s, a, π)

history

[(0, 1, -0.1)]

In [ ]:
# In the simulation above, we have 3 observations of the triplet (state, action, reward)
# Notice the reward of -0.1. This is because we chose to make additional investments (the units are in millions)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class MDP():
    def __init__(self):
        self.A = [0, 1]  # Actions: no investment, invest
        self.S = [0, 1, 2, 3, 4]  # States

        # Transition probabilities without investment
        P0 = np.array([
            [0.5, 0.15, 0.15, 0, 0.20],
            [0, 0.5, 0.0, 0.25, 0.25],
            [0, 0, 0.15, 0.05, 0.8],
            [0, 0, 0, 0, 1],
            [0, 0, 0, 0, 1]
        ])

        # Rewards without investment
        R0 = np.array([0, 0, 0, 10, 0])

        # Transition probabilities with investment
        P1 = np.array([
            [0.5, 0.25, 0.15, 0, 0.10],
            [0, 0.5, 0.0, 0.35, 0.15],
            [0, 0, 0.20, 0.05, 0.75],
            [0, 0, 0, 0, 1],
            [0, 0, 0, 0, 1]
        ])

        # Rewards with investment (including -0.1 investment cost)
        R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

        self.P = [P0, P1]
        self.R = [R0, R1]

    def step(self, s, a):
        # Simulate next state and reward
        s_prime = np.random.choice(len(self.S), p=self.P[a][s])
        R = self.R[a][s]
        done = (s_prime == 4)
        return s_prime, R, done

    def monte_carlo_evaluation(self, policy, num_episodes=10000, discount_factor=0.9960):
        # Initialize value function estimate
        V = np.zeros(len(self.S))
        returns = [[] for _ in range(len(self.S))]

        for _ in range(num_episodes):
            # Start at initial state
            s = 0
            history = []
            done = False

            # Generate episode
            while not done:
                # Choose action based on policy
                a = policy[s]

                # Take step
                s_prime, R, done = self.step(s, a)

                # Store experience
                history.append((s, a, R))
                s = s_prime

            # Calculate returns
            G = 0
            for t in range(len(history)-1, -1, -1):
                s, a, R = history[t]
                G = R + discount_factor * G
                returns[s].append(G)

        # Calculate average returns
        for s in range(len(self.S)):
            if returns[s]:
                V[s] = np.mean(returns[s])

        return V

# Create MDP instance
mdp = MDP()

# Policy A: Always invest
policy_always_invest = [1, 1, 1, 0, 0]
V_always_invest = mdp.monte_carlo_evaluation(policy_always_invest)

# Policy B: Never invest
policy_never_invest = [0, 0, 0, 0, 0]
V_never_invest = mdp.monte_carlo_evaluation(policy_never_invest)

print("Value Function - Always Invest:")
for s, v in enumerate(V_always_invest):
    print(f"State {s}: {v:.4f}")

print("\nValue Function - Never Invest:")
for s, v in enumerate(V_never_invest):
    print(f"State {s}: {v:.4f}")

Value Function - Always Invest:
State 0: 3.4221
State 1: 6.8745
State 2: 0.5052
State 3: 10.0000
State 4: 0.0000

Value Function - Never Invest:
State 0: 1.6242
State 1: 4.9826
State 2: 0.5355
State 3: 10.0000
State 4: 0.0000
